In [36]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import pickle

In [40]:
# import os
# # Using OS library to call CLI commands in Python
# os.system("snscrape --jsonl --max-results 5000 --since 2020-06-01 twitter-search 'business until:2020-06-30' > June_text-query-tweets_biz.json")

class TweetScrapper():
    
#     def __init__(self):
#         raw_tweet_df = []
        
    def scrap(self, filename):
        dates = pd.read_csv('./data/dates_sm.csv')

        tweets=[]
        tweets_economics=[]
        tweets_biz=[]
        tweets_finance=[]
        for idx, row in dates.iterrows():
            dt1, dt2 =row[0].split(' ')

            keyword1 = "economics since:{} until:{}".format(dt1, dt2)
            keyword2 = "business since:{} until:{}".format(dt1, dt2)
            keyword3 = "financ since:{} until:{}".format(dt1, dt2)

            # Using TwitterSearchScraper to scrape data and append tweets to list
            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword1).get_items()):
                if i>1500:
                    break
                tweets_economics.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.lang])
                tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.lang])


            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword2).get_items()):
                if i>1500:
                    break
                tweets_biz.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.lang])
                tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.lang])

            for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword3).get_items()):
                if i>1500:
                    break
                tweets_finance.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.lang])
                tweets.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.likeCount, tweet.retweetCount, tweet.lang])

        # Creating a dataframe from the tweets list above
        tweets_df1= pd.DataFrame(tweets_economics, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'likeCount', 'retweetCount', 'lang'])
        tweets_df2= pd.DataFrame(tweets_biz, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'likeCount', 'retweetCount', 'lang'])
        tweets_df3= pd.DataFrame(tweets_finance, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'likeCount', 'retweetCount', 'lang'])

        tweets_df= pd.DataFrame(tweets, columns=['Datetime', 'Tweet Id', 'Text', 'Username', 'likeCount', 'retweetCount', 'lang'])
        tweets_df.to_csv('./tweet_data/{}'.format(filename), encoding='utf-8', index=False)
        return tweets_df 

    def tweet_to_dataFrame(self, filename):
        raw_tweet_df = pd.read_csv("./tweet_data/{}".format(filename))
        raw_tweet_df["Datetime"] =  pd.to_datetime(raw_tweet_df.Datetime, infer_datetime_format=True)
        raw_tweet_df["Text"] = raw_tweet_df["Text"].astype(str)
        raw_tweet_df['date'] = raw_tweet_df['Datetime'].dt.date
        raw_tweet_df['time'] = raw_tweet_df['Datetime'].dt.time
        raw_tweet_df['month'] = raw_tweet_df['Datetime'].dt.strftime("%m")
        raw_tweet_df['year'] = raw_tweet_df['Datetime'].dt.strftime("%y")
        raw_tweet_df['period'] = raw_tweet_df[['year', 'month']].apply(lambda x: '-'.join(x), axis=1)
        raw_tweet_df = raw_tweet_df[raw_tweet_df.lang == 'en']
        return raw_tweet_df
    
    def pickle_tweets(self, tweets_df, filename):
        outfile = open(filename,'wb')
        pickle.dump(tweets_df,outfile)
        outfile.close()

In [39]:
raw_tweet_df = []

if __name__ == '__main__':
    filename = 'tweets.csv'
    tweet_scrapper = TweetScrapper() 
    tweets_df = tweet_scrapper.scrap(filename)
    raw_tweet_df = tweet_scrapper.tweet_to_dataFrame(filename)
    tweet_scrapper.pickle_tweets(raw_tweet_df, "raw_tweet_pickle.pkl")